In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', optimized=True, numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import collections as col
np.set_printoptions(suppress=True, precision=3)

In [61]:
run_id='icqg'
sample_sizes=[250, 500, 750, 1000]
number_of_qtl = 10
number_of_replicates = 50
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 500
recombination_rates = [0.01]*1478

In [8]:
founder_pair_count = len(founders)

In [16]:
if founder_pair_count == 1:
    pass
elif founder_pair_count % 2 == 0:
    founder_pair_count /= 2
elif founder_pair_count % 2 > 0:
    founder_pair_count + 1
    founders.append(founders[0])
else:
    pass

In [13]:
len(founders) / 2

2.0

In [58]:
founders

[[2, 26], [3, 25], [4, 24], [5, 23]]

In [63]:
founder_generator = (founder_pair for founder_pair in founders)

In [60]:
while founder_generator:
    print(next(founder_generator))

[2, 26]
[3, 25]
[4, 24]
[5, 23]


StopIteration: 

In [64]:
for pair in founder_generator:
    print(pair)

[2, 26]
[3, 25]
[4, 24]
[5, 23]


In [4]:
prefounders = sim.loadPopulation('prefounders1478.pop')
multi_prefounders = sim.Simulator(prefounders, number_of_replicates, stealPops=False)
magic = breed.MAGIC(multi_prefounders, founders, recombination_rates)
sim.tagID(prefounders, reset=27)
magic.generate_f_one(founders, os_per_pair)
mrc = breed.MultiRandomCross(multi_prefounders, 4, os_per_pair)
mother_choices, father_choices = mrc.determine_random_cross()
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(mother_choices, father_choices)
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 1000)
final_mothers, final_fathers = final_mrc.determine_random_cross()
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(final_mothers, final_fathers)
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)
mater = breed.MAGIC(multi_prefounders, founders, [0.01]*1478)
mater.random_mating(3, 2000)

Initiating random mating for 3 generations.


In [65]:
icqg = analyze.Study('icqg')

In [66]:
sample_library = icqg.collect_samples(multi_prefounders, sample_sizes)

In [67]:
sample_library

{0: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 1: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 2: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 3: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 4: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 5: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 6: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 7: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 8: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],
 9: [<simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>,
  <simuPOP.Population>],


In [68]:
alleles = np.array(pd.read_hdf('parameters\\alleles_at_1478_loci.hdf'))

In [69]:
analyze.multi_sample_allele_frq_storage(sample_library, alleles, run_id)

C:\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '250'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '500'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Anaconda3\lib\site-packages\tables\path.py:100: Nat

In [71]:
sets_of_segregating_loci = icqg.seg_loci_among_samples(sample_library)

In [72]:
sets_of_segregating_loci.values()

dict_values([200])

In [73]:
concordant_segregating_loci = list(sets_of_segregating_loci.keys())[0]

In [76]:
qtl = tuple(sorted(random.sample(concordant_segregating_loci, number_of_qtl)))

In [77]:
qtl

(97, 107, 301, 356, 633, 637, 1097, 1169, 1311, 1333)

In [83]:
add_trait = parameters.Trait()

In [84]:
exponential_allele_effects = add_trait.assign_allele_effects(alleles, qtl, np.random.exponential, 1, multiplicity=1)

In [81]:
import importlib as imp
imp.reload(parameters)

<module 'saegus.parameters' from 'C:\\Anaconda3\\lib\\site-packages\\saegus\\parameters.py'>

In [86]:
geometric_allele_effects = add_trait.assign_geometric_series(exponential_allele_effects, 0.90, 10)

In [87]:
geometric_allele_effects

{97: {1: 0.9, 2: 0},
 107: {1: 0.81, 3: 0},
 301: {1: 0.7290000000000001, 3: 0},
 356: {0: 0.6561, 3: 0},
 633: {2: 0.5904900000000001, 3: 0},
 637: {1: 0.531441, 3: 0},
 1097: {1: 0.4782969000000001, 2: 0},
 1169: {0: 0.4304672100000001, 3: 0},
 1311: {1: 0.3874204890000001, 3: 0},
 1333: {4: 0.3486784401000001, 5: 0}}

In [88]:
exponential_allele_effects

{97: {1: 1.051432857038865, 2: 0.057826115322452784},
 107: {1: 2.6121643009302526, 3: 0.6785810010503582},
 301: {1: 0.16514923263694337, 3: 0.40333256089372005},
 356: {0: 0.8057544731217682, 3: 2.668606006563908},
 633: {2: 0.637690579008768, 3: 1.396970469747448},
 637: {1: 1.037086746271281, 3: 0.35723126659288434},
 1097: {1: 1.971178392119941, 2: 2.855233431514642},
 1169: {0: 1.5942361468831883, 3: 0.607884448672498},
 1311: {1: 2.1739729921340385, 3: 0.5465183224694323},
 1333: {4: 0.468064705529327, 5: 1.0394027313631007}}

In [90]:
concordant_segregating_loci

(1,
 2,
 3,
 4,
 5,
 6,
 8,
 10,
 12,
 13,
 14,
 15,
 16,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 29,
 30,
 31,
 32,
 34,
 36,
 37,
 38,
 39,
 40,
 42,
 45,
 46,
 50,
 52,
 53,
 54,
 56,
 59,
 61,
 62,
 63,
 66,
 67,
 68,
 70,
 71,
 74,
 76,
 77,
 79,
 80,
 82,
 86,
 88,
 90,
 91,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 104,
 106,
 107,
 108,
 109,
 114,
 115,
 116,
 119,
 120,
 121,
 122,
 123,
 124,
 126,
 127,
 128,
 129,
 130,
 132,
 134,
 136,
 138,
 139,
 140,
 141,
 144,
 145,
 146,
 147,
 149,
 152,
 153,
 155,
 158,
 159,
 163,
 165,
 168,
 170,
 171,
 172,
 173,
 174,
 175,
 177,
 181,
 182,
 185,
 186,
 190,
 191,
 192,
 193,
 196,
 198,
 202,
 208,
 209,
 211,
 212,
 214,
 215,
 218,
 219,
 220,
 221,
 223,
 224,
 225,
 227,
 229,
 232,
 235,
 240,
 241,
 246,
 247,
 249,
 250,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 261,
 262,
 263,
 264,
 265,
 266,
 268,
 270,
 272,
 274,
 275,
 277,
 278,
 279,
 280,
 282,
 283,
 284,
 285,
 288,
 291,
 292,
 293,
 294,
 297,
 

In [91]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage[run_id] = concordant_segregating_loci
seg_loc_storage.close()

In [92]:
droppable_loci = [droppable for droppable in range(1478) if droppable not in concordant_segregating_loci]

In [93]:
droppable_loci

[0,
 7,
 9,
 11,
 17,
 18,
 19,
 22,
 28,
 33,
 35,
 41,
 43,
 44,
 47,
 48,
 49,
 51,
 55,
 57,
 58,
 60,
 64,
 65,
 69,
 72,
 73,
 75,
 78,
 81,
 83,
 84,
 85,
 87,
 89,
 92,
 93,
 94,
 102,
 105,
 110,
 111,
 112,
 113,
 117,
 118,
 125,
 131,
 133,
 135,
 137,
 142,
 143,
 148,
 150,
 151,
 154,
 156,
 157,
 160,
 161,
 162,
 164,
 166,
 167,
 169,
 176,
 178,
 179,
 180,
 183,
 184,
 187,
 188,
 189,
 194,
 195,
 197,
 199,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 210,
 213,
 216,
 217,
 222,
 226,
 228,
 230,
 231,
 233,
 234,
 236,
 237,
 238,
 239,
 242,
 243,
 244,
 245,
 248,
 251,
 260,
 267,
 269,
 271,
 273,
 276,
 281,
 286,
 287,
 289,
 290,
 295,
 296,
 298,
 306,
 309,
 320,
 322,
 323,
 324,
 325,
 326,
 330,
 332,
 337,
 340,
 343,
 347,
 351,
 357,
 358,
 360,
 364,
 365,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 376,
 377,
 385,
 386,
 391,
 396,
 397,
 403,
 405,
 406,
 407,
 409,
 414,
 415,
 416,
 417,
 419,
 422,
 424,
 425,
 427,
 439,
 444,
 446,
 455,
 45

In [101]:
imp.reload(analyze)

<module 'saegus.analyze' from 'C:\\Anaconda3\\lib\\site-packages\\saegus\\analyze.py'>

In [103]:
analyze.store_allele_effect_frequency_tables(sample_library, alleles, 
                                             qtl,
                                             exponential_allele_effects,
                                            run_id, 'exponential')

C:\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '250'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '500'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Anaconda3\lib\site-packages\tables\path.py:100: Nat

In [ ]:
analyze.generate_allele_effects_frequencies()

In [ ]:
store_allele_effect_frequency_tables(sample_library, ge, alpha_alleles, beta_alleles, run_id, sub_run_id)

In [ ]:
expanded_aef = assign_alpha_beta_allele_frequencies(rep, ge, alpha_alleles, beta_alleles)

In [ ]:
multi_sample_allele_effect_and_frq = pd.HDFStore('dragonlance_geometric_allele_effects_and_frequencies.h5')

In [ ]:
multi_sample_allele_effect_and_frq.close()

In [ ]:
multi_sample_allele_effect_and_frq.open()

In [ ]:
multi_sample_allele_effect_and_frq.close()

In [ ]:
sub_run_id = 'geometric'

In [ ]:
'/'.join([run_id, sub_run_id])

In [ ]:
for rep_id, samples in sample_library.items():
    for sample in samples:
        expanded_allele_effects_and_frequencies = generate_allele_effects_frequencies(sample, ge, alpha_alleles, beta_alleles)
        name = '/'+ '/'.join([run_id, sub_run_id, str(rep_id), str(sample.popSize())])
        multi_sample_allele_effect_and_frq.put(name, expanded_allele_effects_and_frequencies)
        

In [ ]:
imp.reload(analyze)

In [ ]:
sub_run_id = 'geometric'

In [ ]:
su = analyze.generate_super_table(run_id, 1, 500, concordant_segregating_loci, sub_run_id=sub_run_id)

In [ ]:
pd.HDFStore('dragonlance_storage.h5')

In [ ]:
dir()

In [ ]:
del example_af

In [ ]:
del reloaded_table

In [ ]:
geometric_series = []
a = 9/10
for i in range(1, 11):
    geometric_series.append(a**i)

In [ ]:
(9/10)**1

In [ ]:
geometric_series

In [ ]:
for locus in alleles:
    alpha_allele, beta_allele = alleles[locus]
    

In [ ]:
alpha_alleles = alleles[:, 0]

In [ ]:
beta_alleles = alleles[:, 1]

In [ ]:
beta_alleles

In [ ]:
geometric_allele_effects

In [ ]:
def assign_geometric_series_effects(base, power):
    geometric_series = [base**i_power for i_power in range(1, power+1)]
    return geometric_series

In [ ]:
geffects = col.defaultdict(float)

In [ ]:
alpha_qt_alleles = [alpha_alleles[locus] for locus in qtl]

In [ ]:
alpha_qt_alleles

In [ ]:
geffects

In [ ]:
exponential_allele_effects

In [ ]:
import copy

In [ ]:
geometric_ae = copy.deepcopy(exponential_allele_effects)

In [ ]:
geometric_ae[2][1] = geometric_series[0]
geometric_ae[2][3] = 0

In [ ]:
geometric_ae = copy.deepcopy(exponential_allele_effects)

In [ ]:
geometric_ae

In [ ]:
def assign_geometric_series(allele_effects, geometric_series):
    geometric_allele_effects = copy.deepcopy(allele_effects)
    for locus, i in zip(sorted(allele_effects.keys()), range(len(geometric_series))):
        ael = list(allele_effects[locus])
        geometric_allele_effects[locus][ael[0]] = geometric_series[i]
        geometric_allele_effects[locus][ael[1]] = 0
    return geometric_allele_effects

In [ ]:
geometric_series

In [ ]:
ge = assign_geometric_series(exponential_allele_effects, geometric_series)

In [ ]:
ge

In [ ]:
ge_table = analyze.generate_allele_effects_table(qtl, alleles, ge, saegus_to_tassel_loci)

In [ ]:
columns = ['locus', 'tassel_locus', 'alpha_allele', 'alpha_frequency', 'alpha_effect',
          'beta_allele', 'beta_frequency', 'beta_effect', 'difference']

In [ ]:
expanded_aetable = pd.DataFrame(np.zeros((1478, 9)), columns=columns)

In [ ]:
for locus, ae_map in ge.items():
    expanded_aetable.ix[locus, 'alpha_effect']

In [ ]:
for i, allele_set in enumerate(alleles):
    

In [ ]:
expanded_aetable.ix[:, 'alpha_allele'] = alpha_alleles

In [ ]:
expanded_aetable.ix[:, 'beta_allele'] = beta_alleles

In [ ]:
af.join(expanded_aetable)

In [ ]:
np.array(af['major_allele'], dtype=np.int64)

In [ ]:
np.where(False, np.equal(np.array(expanded_aetable['alpha_allele']), np.array(af['major_allele'], dtype=np.int64)))

In [ ]:
sample_library[0][0].dvars().alleleFreq

In [ ]:
ge

In [ ]:
for 

In [ ]:
1478 - 1429

In [ ]:
dict(af)

In [ ]:
for locus, alpha_allele in enumerate(alpha_alleles):
    af[]

In [ ]:
alpha_allele_frequencies = 

In [ ]:
expanded_aetable

In [ ]:
del geometric_allele_effects

In [ ]:
for locus, allele_effect_map in geometric_ae.items():
    ael = list(allele_effect_map.keys())    
    geometric_allele_effects[locus][ael[0]]

In [ ]:
geometric_ae

In [ ]:
alle

In [ ]:
for locus in qtl:
    alpha_allele = alpha_alleles[locus]
    geffects[locus]

In [ ]:
assign_geometric_series_effects(9/10, 10)

In [ ]:
geometric_allele_effects = col.defaultdict

In [ ]:
def_allele_effects = col.defaultdict(geometric_allele_effects, 0.0, float)

In [ ]:
alph

In [ ]:
alpha_allele_effects = allele_effects

In [ ]:
alpha_allele, beta_allele = allel

In [ ]:
geometric_allele_effects_table

In [ ]:
af = allele_frequencies['/dragonlance/0/1000']

In [ ]:
af.ix[:10, ]

In [ ]:
af['minor_allele'][0]

In [ ]:
af['minor_allele'][0] == 2.0 or af['major_allele'][0] == 2.0

In [ ]:
af.minor_allele[0]

In [ ]:
if af.minor_allele[0] == 

In [ ]:
alpha_alleles = [allele for allele in geometric_allele_effects_table['alpha_allele']]

In [ ]:
alpha_alleles

In [ ]:
beta_alleles = af['minor_allele']

In [ ]:
beta_alleles

In [ ]:
allele_effect

In [ ]:
alpha_alleles = 

In [ ]:
for allele in geometric_allele_effects_table['alpha_allele']:
    print(allele)

In [ ]:
np.zeros(())

In [ ]:
allele_effects_table = analyze.generate_allele_effects_table(qtl, geometric_allele_effects, b
                                                            )

In [ ]:
def remap_afrq_table_loci(allele_frequency_table,
                          saegus_to_tassel_loci):

    loci = list(allele_frequency_table.index)
    segregating_loci = list(saegus_to_tassel_loci.keys())
    droppable_loci = [locus for locus in loci if locus not in segregating_loci]
    allele_frequency_table_subset = allele_frequency_table.drop(droppable_loci,
                                                                axis=0)
    new_index = list(range(len(segregating_loci)))
    allele_frequency_table_subset.index = new_index
    return allele_frequency_table_subset

In [ ]:
cd C:\Users\DoubleDanks\BISB\wisser\code\rjwlab-scripts\saegus_project\devel\magic\1478

In [ ]:
allele_frequencies = pd.HDFStore('dragonlance_storage.h5', mode='r')

In [ ]:
allele_frequenciesrequencies

In [ ]:
example_af = allele_frequencies['/dragonlance/0/500']

In [ ]:
example_af.ix[:10, ]

In [ ]:
j.ix[:10,]

In [ ]:
j = rcon.join(example_af)

In [ ]:
example_af.to_csv('exampleaf.txt', sep='\t')

In [ ]:
j.to_csv('joined.txt', sep='\t')

In [ ]:
subset_geo_af = remap_afrq_table_loci(example_af, saegus_to_tassel_loci)

In [ ]:
subset_geo_af.to_csv('subsetaf.txt', sep='\t')

In [ ]:
pd.read_csv('dragonlance_geometric_0_500_qvalues.txt', sep='\t')

In [ ]:
rcon = analyze.reconfigure_gwas_results('dragonlance_geometric_0_500_out_2.txt', 'dragonlance_geometric_0_500_qvalues.txt', geo_expanded)

In [ ]:
rcon.join(example_af)

In [ ]:
geometric_allele_effects_table

In [ ]:
supert = analyze.generate_super_table('dragonlance_geometric_0_500_out_2.txt', 'dragonlance_geometric_0_500_out_1.txt', example_af, geo_expanded)

In [ ]:
def generate_super_table(gwas_results_file, q_values_file,
                         subsetted_allele_frequency_table,
                         expanded_allele_effects_table, delim="\t"):


    """
    Combines the TASSEL output with allele frequencies, allele effects and
    q values.


    """
    gwas_results = pd.read_csv(gwas_results_file, sep=delim)
    gwas_results.drop('Trait', axis=1, inplace=True)
    gwas_results.drop('Pos', axis=1, inplace=True)
    gwas_results.drop(0, axis=0, inplace=True)
    gwas_results = gwas_results.ix[:, 'Marker':'dom_p']
    gwas_results.index = gwas_results.index - 1
    gwas_results.drop('Marker', axis=1, inplace=True)
    qvalues = pd.read_csv(q_values_file, sep=delim)
    qvalues.columns = ['q']
    qvalues.index = qvalues.index - 1
    results = gwas_results.join(qvalues)
    greater_results = results.join(expanded_allele_effects_table)
    super_table = greater_results.join(subsetted_allele_frequency_table)

    return super_table

In [ ]:
    gwas_results = pd.read_csv(gwas_results_file, sep=delim)
    gwas_results.drop('Trait', axis=1, inplace=True)
    gwas_results.drop('Pos', axis=1, inplace=True)
    gwas_results.drop(0, axis=0, inplace=True)
    gwas_results = gwas_results.ix[:, 'Marker':'dom_p']
    gwas_results.index = gwas_results.index - 1
    gwas_results.drop('Marker', axis=1, inplace=True)
    qvalues = pd.read_csv(q_values_file, sep=delim)
    qvalues.columns = ['q']
    qvalues.index = qvalues.index - 1

In [ ]:
geo_expanded

In [ ]:
allele_effects_store = shelve.open('allele_effects_storage')
allele_effects_store['dragonlance_exponential'] = exponential_allele_effects
allele_effects_store['dragonlance_geometric'] = geometric_allele_effects
allele_effects_store.close()

In [ ]:
loci_conversions = shelve.open(run_id+'_loci_conversions')
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(concordant_segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus
loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci
loci_conversions.close()

In [ ]:
exponential_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                exponential_allele_effects, saegus_to_tassel_loci)

In [ ]:
geometric_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                geometric_allele_effects, saegus_to_tassel_loci)

In [ ]:
exponential_allele_effects_table

In [ ]:
geometric_allele_effects_table

In [ ]:
import importlib as imp
imp.reload(analyze)

In [ ]:
cd C:\Users\DoubleDanks\BISB\wisser\code\rjwlab-scripts\saegus_project\devel\magic\1478

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       exponential_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_exponential',
                                       allele_frequency_hdf=run_id+'_storage.h5')

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       geometric_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_geometric', 
                                       allele_frequency_hdf=run_id+'_storage.h5')

In [ ]:
import ipyparallel as ipp

In [ ]:
c = ipp.Client()

In [ ]:
cd C:\tassel\output

In [ ]:
exp_expanded = analyze.remap_ae_table_loci(exponential_allele_effects_table, saegus_to_tassel_loci)

In [ ]:
geo_expanded = analyze.remap_ae_table_loci(geometric_allele_effects_table, saegus_to_tassel_loci)

In [ ]:
dragonlance = analyze.Study(run_id+'_geometric')

In [ ]:
expo_fpr_raw_data = dragonlance.collect_power_analysis_data(sample_sizes, number_of_replicates, exp_expanded)

In [ ]:
cd C:\tassel\output

In [ ]:
geometric_fpr_raw_data = dragonlance.collect_power_analysis_data(sample_sizes, number_of_replicates, geo_expanded)

In [ ]:
ls

In [ ]:
exp_results, exp_true_positives, exp_false_positives = dragonlance.calculate_power_fpr(expo_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
geo_results, geo_true_positives, geo_false_positives = demonstration.calculate_power_fpr(geo_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
qtl

In [ ]:
saegus_to_tassel_loci[2]

In [ ]:
popgen = (pop for pop in sample_library[0])

In [ ]:
for pop in popgen:
    